# Sorting the travelling salesman problem

Enrique Pérez Arnaud

## Abstract

In this paper I use *machine learning* (ML) to produce some evidence supporting the possibility of finding a polynomial time algorithm for solving instances of the *travelling salesman problem* (TSP).

The decision we ask the ML models to make is as follows.
First we create a random distribution of points in the plane;
a distribution that the model has never seen.
Then we look for the shortest possible circuit of the points that we can find,
which is not necessarily the solution to the instance of the problem;
and then we ask the model whether the circuit is actually the shortest,
i.e., whether it is the solution,
or whether there may be some shorter circuit that we have not found.
It is a binary classification task, i.e.,
we don't ask the model for a shorter circuit.

Our models are producing (percentage) confusion matrices like
$\left[\begin{smallmatrix}61 & 39 \\\\ 26 & 74\end{smallmatrix}\right]$, which, to me, seem significative.

Note that the evidence shown is not conclussive,
mainly beacuse the data used in the models is taken from instances of the problem with, at most, 31 cities.

To understand what kind of data we are feeding the ML models,
and the significance of the predictions,
I provide an introduction in which the TSP is approached from the perspective of the *sorting problem* (SP) - i.e.,
the problem that is solved by sorting algorithms.
Note that both problems have the same basic form:
we want to choose one among all the permutations of a set of elements, according to some criterion.
But, whereas the SP can be solved in fairly efficient ways, the TSP cannot.
Thus we want to understand the difference between both problems
that is the basis for the difference in efficiency of the algorithms we use to solve them.
We want to understand the precise shape of the input data to both problems,
and the characteristics of the data that are present in instances of the SP
and absent in instances of the TSP,
so that we can provide our ML models with sufficient and appropriately shaped data.

### The sorting problem

As said above, what we refer to here as the SP is the problem that is solved by sorting algorithms.
For simplicity, here I will only consider sorting lists with no repeated elements,
but the argument might easily be transformed to cover lists with repeated elements.

There are 2 sides to the SP:
The first is to get to the solution,
and the second is to get there efficiently.
So the first step here will be to formalize the first side of the problem:
what exactly is an instance of the problem, and what its solution.
This formalization is geared towards extending it to cover the TSP,
so it is not necessarily the most natural formalization of the SP.
Later, a second step will be to examine the different algorithms
(and their efficiencies)
that we can use to go from an instance of the problem to its solution.

I will start with a basic definition. Given a finite set $\mathbf{S}$, with $\mathbf{c} = |\mathbf{S}|$ being its cardinality, an *index* on $\mathbf{S}$ is a bijective map from $[1..\mathbf{c}] \subset \mathbb{N}$ to $\mathbf{S}$. We denote by $\mathbf{I}_\mathbf{S}$ the set of indexes on $\mathbf{S}$:

$$
\mathbf{I}_\mathbf{S} = \{ \mathbf{i}: [1..\mathbf{c}] \rightarrow \mathbf{S} \}
$$

Given this, we can specify an *instance of the SP* (an ISP) providing a set $\mathbf{S}$ and an index $\mathbf{i}$ on it,
where $\mathbf{S}$ is a finite subset of some universal set $\mathbf{U}$ from which it inherits
a total order relation $\le_\mathbf{S}$.

The solution to this ISP that we are asked for is another particular index $\mathbf{j} \in \mathbf{I}_\mathbf{S}$ that is monotonic wrt $\le_\mathbf{S}$.

Since we want constructible elements in $\mathbf{U}$,
so that we can fully provide them in the specification of an ISP,
and at the same time we want the most freedom in choosing $\mathbf{S}$,
we can assume $\mathbf{U}$ to be here the set of rationals, $\mathbb{Q}$,
and $\le_\mathbf{S}$ the ususal $\le$ in $\mathbb{Q}$.

### Graph theoretic picture of the SP

At this point, it is convenient to provide a graph theoretic picture of the structure at hand.
We can define here a directed graph $\Psi_\mathbf{S}$, in which each node is a permutation of $\mathbf{S}$,
taken as a pair of linear orderings on $\mathbf{S}$,
one given by an index $\mathbf{i}$ and the other given by $\le_\mathbf{S}$,
so each node can be represented by its index;
and in which each directed edge is also a permutation,
this time taken as an automorphism on $\mathbf{S}$.
This is a total graph, in which there is an edge from any node to any other node.
We will denote here the set of vertices of $\Psi_\mathbf{S}$ by $\mathbf{V}[\Psi_\mathbf{S}]$, and its set of edges by $\mathbf{E}[\Psi_\mathbf{S}]$.

In this graph picture, an ISP is given by any arbitrary node $\mathbf{i} \in \mathbf{V}[\Psi_\mathbf{S}]$. We will use the same symbol to refer to a node and to its index.

### Solving the SP - naïve algorithm

Given all this, we can provide a criterion, a function,
that given an index on a finite set $\mathbf{S}$ on which there is a total order relation,
decides whether it is monotonic wrt that order relation - whether it is ordered.

First we build a function $\mathbf{lt} : \mathbb{Q}^2 \rightarrow \{0,1\}$ that tels us whether 2 elements are ordered:

$$
\mathbf{lt}(a,b) \mapsto \begin{cases} 0 & \text{if } a > b \\ 1 & \text{if } a \le b \end{cases}
$$

Then we build a function $\mathbf{O} : \mathbf{I}_\mathbf{S} \rightarrow \{0,1\}$ that tells us whether an index on $\mathbf{S}$ is monotonic:

$$
\mathbf{O}(\mathbf{i}) \mapsto \prod_{n=1}^{\mathbf{c}-1} \mathbf{lt}(\mathbf{i}(n), \mathbf{i}(n+1))
$$

So now we can devise a simple algorithm to obtain a solution given any ISP:
Construct the set of all permutations of the index provided in the spec of the problem,
and check each for monotonicity, and choose the one that is monotonic.

[Here](Python%20-%20SP%20brute%20force.ipynb) can be found a demonstration of this algorithm in Python,
that shows how phenomenally inefficient it is.

### Graph theoretic picture of the naïve algorithm to solve the SP

To understand this algorithm in graph terms,
we start from the total graph $\Psi_\mathbf{S}$, and a distinguished node $\mathbf{i}$ -
the one given by the index provided in the spec of the ISP.
We have in this graph a lot of edges, and we want to choose one,
that takes us from $\mathbf{i}$ to the solution $\mathbf{j}$.
We start by discarding all edges $\mathbf{e} \in \mathbf{E}[\Psi_\mathbf{S}]$ whose source $\mathbf{src}(\mathbf{e}) \neq \mathbf{i}$.
Then, we give a weight to each of the edges $\mathbf{e}$ with $\mathbf{src}(\mathbf{e}) = \mathbf{i}$:
we assign 1 when the index of the destination node $\mathbf{dst}(\mathbf{e})$ of the edge is monotonic, and -1 otherwise.
Finally we discard all the nodes with weight -1,
and we are left with a single edge with source in the distinguished node and destination in the solution node (assuming, as said above, that there are no repeated elements in $\mathbf{S}$).

### Towards better algorithms

Now we get to the 2nd part of the problem: can we do better (in terms of resources needed for the computation)?

The problematic thing we've done above,
wrt computation cost,
is to gather all permutations of the given index of $\mathbf{S}$.
The set $\mathbf{P}_\mathbf{S}$ of permutations of $\mathbf{S}$,
in general, is very big - $|\mathbf{P}_\mathbf{S}| = \mathbf{c}!$.
But, in addition, this set can be given the structure of a group,
which means that we can have a very small set
$\mathbf{G}_\mathbf{S} \subset \mathbf{P}_\mathbf{S}$ of generators for the group,
with the guarantee that we can go from any element of the group (any index on $\mathbf{S}$) to any other
by multiplying it with some sequence of elements from this set of generators.

Note that Ps and Is are isomorphic as (non abelian) groups; with concatenation as the group operation for Is, and application for Ps.

So in the naïve method, given an element p in Ps, we check all the other elements of Ps to see whether they combine with p to produce a monotonic ordering of S. Restricting ourselves to a generator set Gs, we want to find a sequence of elelements of Gs that, combined with p, produce a monotonic ordering of S.

In graph theoretic terms,
what we are doing is taking a subgraph $\Phi_{\mathbf{S},\mathbf{G}_\mathbf{S}} \leq \Psi_\mathbf{S}$,
in which we remove all edges in $\mathbf{E}[\Psi_\mathbf{S}]$ that do not correspond
to elements of the set of generators.

$$
\mathbf{E}[\Phi_{\mathbf{S},\mathbf{G}_\mathbf{S}}] = \{\mathbf{e} \in \mathbf{E}[\Psi_\mathbf{S}] : \mathbf{p}(\mathbf{e}) \in \mathbf{G}_\mathbf{S}\}
$$

Where $\mathbf{p}(\mathbf{e})$ denotes the permutation in $\mathbf{P}_\mathbf{S}$ that corresponds to the given edge $\mathbf{e}$.


now we can go from any node to any other node, but following a (directed) path rather than just an edge.

### Generator sets

There are many kinds of generator sets,
and we are not going here to try to provide a systematic clasification of them.
We are just going to define a class of them,
that will be useful for our purposes.
This class is based on a special permutation,
which we will call a *roller permutation*, denoted by $\mathbf{r}_\mathbf{c} \in \mathbf{P}_\mathbf{S}$
given that $\mathbf{c} = |\mathbf{S}|$:

$$
\mathbf{r}_\mathbf{c} = (2,3, ... \mathbf{c}-1,\mathbf{c},1)
$$

Given this, we will call a generator set *rollable* if applying  $\mathbf{r}_\mathbf{c}$ to all its elements
results in the same unchanged generator set.

So, for example, the simplest generator set for a symmetric group, given by the roller permutation and the permutation that swaps the 1st 2 elements, would not be rollable,
but the generator set consisting of $\mathbf{c}$ elements, each swapping 2 different consecutive elements,
would be rollable.

[Here](generators.py) we can find a Python module
that will allow us to construct different (rollable) generator sets,
in the form of lists of permutation matrices.

### Compass functions

Choosing a generator set can help wrt performance, but not if we keep our test - monotonicity.
This test has a boolean value, and is only true on the full solution.
Any other index is just false;
this means that we need a full path on $\Phi_{\mathbf{S},\mathbf{G}_\mathbf{S}}$ to know whether it leads to the solution.
What we want is a test that we can apply at each node in the graph,
so we can build a path step by step.
A kind of compass to navigate the graph,
so that at each node,
we only have to consider as possible next nodes
those provided by $\mathbf{G}_\mathbf{S}$.

Definition: A compass function $\mathcal{c} \in \mathcal{C}_\mathbf{S}$ is a map from the set of indexes $\mathbf{I}_\mathbf{S}$ of some $\mathbf{S}$ to $\mathbb{Q}$.

$$
\mathcal{C}_\mathbf{S} = \{\mathcal{c} : \mathbf{I}_\mathbf{S} \rightarrow \mathbb{Q}\}
$$

Regarding graphs, a compass function $\mathcal{c}$ allows us to define
a weight function $\mathbf{w}_\mathcal{c}: \mathbf{E}[\Phi_{\mathbf{S},\mathbf{G}_\mathbf{S}}] \rightarrow \mathbb{Q}$ on the edges of the graph:

$$
\mathbf{w}_\mathcal{c}(\mathbf{e}) \mapsto \mathcal{c}(\mathbf{dst}(\mathbf{e})) - \mathcal{c}(\mathbf{src}(\mathbf{e}))
$$

This allows us to further refine $\Phi_{\mathbf{S},\mathbf{G}_\mathbf{S}}$ to only include edges with a negative weight.
Let's call this new graph $\Omega_{\mathbf{S},\mathbf{G}_\mathbf{S},\mathcal{c}} \leq \Phi_{\mathbf{S},\mathbf{G}_\mathbf{S}}$.
Note that we can no longer go from any node to any other node following a directed path; there is no possible path from a node $\mathbf{i}$ to a node $\mathbf{j}$ if $\mathcal{c}(\mathbf{i}) \lt \mathcal{c}(\mathbf{j})$.

### Sinks and solutions

These $\Omega$ graphs will have sinks; nodes with only incident edges.
This means that, starting from any node,
and iteratively building a path by randomly choosing edges with source in the current node,
we will end up in a sink.

So we need compass functions that produce $\Omega$ graphs with sinks in one to one correpondence with our solutions.
Then, to get to the solution(s), starting from any node,
we just need to build a path, edge by edge, until we reach such a sink.

Note that at this point we are talking about more general problems than sorting.
We can have compass functions that produce sinks that correspond with monotonicity,
to solve ISPs,
or that correspond to other kinds of structure to solve other kinds of problem.

### Compass functions for the SP: monotonicity compass

Focusing now on the SP, lets see what compass functions can we come up with
that will produce a sink(s) corresponding to the monotonic index.

Note that we can take the ordering provided by an index $\mathbf{i}$ on $\mathbf{S}$
as a vector in the $\mathbb{Q}^\mathbf{c}$ space,
and we can also take $[1..\mathbf{c}]$ as a vector in that space.
Then, one compass function we can clearly use,
is the distance (or better, to remain within $\mathbb{Q}$, the quadrance)
between both vectors.
Let's call this compass function $\mathcal{m} \in \mathcal{C}$.
Since $[1..\mathbf{c}]$ is isomorphic (as totally ordered structures) to the solution,
the distance between both vectors will be minimal when the index $\mathbf{i}$ corresponds to the solution.

Given an index $\mathbf{i} \in \mathbf{I}_\mathbf{S}$:

$$
\mathcal{m}(\mathbf{i}) \mapsto \sum_{n = 1}^\mathbf{c} (\mathbf{i}(n) - n)^2
$$

We can now use $\mathcal{m}$ to look for a solution to an ISP, step by step:
for each step, choose the generator that most decreases $\mathcal{m}$,
(or, alternatively, choose any generator that decreases $\mathcal{m}$),
and keep doing so until reaching a sink, which will correspond to the solution.

Obviously, we might have a problem if our compass function determines more than one sink,
where some of them do not correspond to solutions.

Conjecture: the $\Omega$ graph of any ISP
given by a rollable set of generators and the $\mathcal{m}$ compass function
has a single sink, corresponding to the solution.

The many sorting algorithms available all essentially correspond to choosing different sets of generators
and different ways of using $\mathcal{m}$ to choose
the next generator to apply when constructing a path to the solution.
Note that considering which of 2 numbers $\mathcal{a}$ and $\mathcal{b}$ is greater
is essentially the same as considering which of
$(\mathcal{a} - 1)^2 + (\mathcal{b} - 2)^2$ and  $(\mathcal{b} - 1)^2 + (\mathcal{a} - 2)^2$
is greater.

[Here](Python%20-%20SP%20monotonic%20compass.ipynb) can be found a Python demonstration
of a generic sorting algorithm that will use this compass function $\mathcal{m}$ and any set of generators.
It is easy to check that, for the rollable sets of generators provided [here](generators.py),
the conjecture seems to hold, i.e., the algorithm always finds the solution.

### Compass functions for the SP: circuit distance compass

Let's now choose a different compass function $\mathcal{d} \in \mathcal{C}$. Given an index $\mathbf{i} \in \mathbf{I}_\mathbf{S}$:

$$
\mathcal{d}(\mathbf{i}) \mapsto \sum_{n = 2}^\mathbf{c} (\mathbf{i}(n) - \mathbf{i}(n - 1))^2
$$

As is easy to check, this compass function would provide an $\Omega$ graph with multiple sinks,
most of which do not correspond to the solution.

[Here](Python%20-%20circuit%20distance%20compass.ipynb) can be found a Python demonstration
of a generic algorithm that will use this compass function $\mathcal{d}$ and any set of generators.
It is not a sorting algorithm, since it will usually hit sinks that do not correspond to the solution.

Note now that this compass function is basically
the function that would tell us the square of the length of a circuit of the cities
in an instance of the TSP if we assume that the cities are laid out in a single line,
and that the elements of $\mathbf{S}$ provide the location of each city in that line.

This gives us entrance to consider the TSP.

## The TSP

We can take the TSP as an SP in which, instead of starting from a subset $\mathbf{S} \subset \mathbb{Q}$,
we start from a subset $\mathbf{S} \subset \mathbb{Q}^2$.
So each element of $\mathbf{S}$ is a rational point on the euclidean plane
(this could be easily generalizable to any other 2d metric space).

In this case, we do not have a natural order relation for $\mathbf{S}$;
to construct a compass function like the provided $\mathcal{m}$ for the SP,
i.e., to be able to measure the distance to a vector isomorphic to the solution
(isomorphic wrt whatever structure is needed;
in the case of the SP, isomorphic would just mean monotonic),
we would need a very precise ordering of 2d points, i.e.,
we'd need the solution.
Note that whereas in the SP all solutions to all instances of the problem are isomorphic to the same structure,
i.e. $[1..\mathbf{c}]$,
in the TSP the solution to each instance of the problem is isomorphic to a very specific sequence of points.

The only 2d distribution of points that might be distinguished
in the same sense that a monotonic 1d sequence is,
would be one in which the points are arranged in a circle and have all
the same distance to the 2 nearest points.

But if we try to use the distance to this distribution as an $\mathcal{m}$ compass function,
we soon see that it does not work: it favors star shaped circuits over the solutions.

[Here](Python%20-%20TSP%20circle.ipynb) there is some Python code that demonstrates this fact.

So in the TSP we are in principle restricted to use something like the $\mathcal{d}$ compass function,
i.e., a measure of the length of the circuit.
And, again, it is easy to see that such a compass function,
in conjunction with sensibly small sets of generators,
results in an $\Omega$ graph
with many sinks, most of which do not correspond to the solution.

For TSP instances, the index functions will return pairs of numbers, i.e., elements of $\mathbb{Q}^2$.
Let's assume that, if $\mathbf{i}$ is an index on $\mathbf{S}$,
$\mathbf{i}_1(n)$ denotes the first component of the pair,
and $\mathbf{i}_2(n)$ the second. Then $\mathcal{d}$ is given by:

$$
\mathcal{d}(\mathbf{i}) \mapsto \sum_{n = 2}^\mathbf{c} \left( \left(\mathbf{i}_1(n) - \mathbf{i}_1(n - 1) \right)^2 + \left(\mathbf{i}_2(n) - \mathbf{i}_2(n - 1) \right)^2 \right)
$$

(Here we are somewhat simplifying the function, to forget the distance between the last and first point;
simply to avoid muddying the formula with modulos).

So, now, the question becomes:
Can we find a different compass function, let's call it $\mathcal{h}$,
that provides an $\Omega$ graph with only the sinks
that correspond to the solutions?
We'd be ok if in this graph there were orphaned nodes,
with no edges;
if the instance of the problem starts from a node, an index,
that is an orphan, we can always shuffle the index,
until we obtain a non orphan node.

If such a compass function $\mathcal{h}$ exists
it should be possible to somehow distinguish,
among all the sinks provided by the $\mathcal{d}$ compass function,
the one that corresponds to the solution from all the others,
just using local data, i.e.,
data which is local to each node in the graph.
This is because the $\mathcal{h}$ compass function should be able
to extract a quantity from the data characterizing each node,
(quantity that should be different from just the circuit length,)
that should be extreme in the nodes that provide a path to the solution sink.
Therefore, the solution sink itself should be extreme in this quantity,
in contrast to the non-solution sinks,
and so, the data characterizing the sinks
should allow us to distinguish between both types of sinks,
solutions and non-solutions.

Now, what is this data that characterizes the nodes?

In the SP, we were characterizing nodes and edges of the graphs just by permutations of the $\mathbf{S}$ sets;
that was all the data needed to restrict the graphs and navigate them.
In the TSP, we need the data in a different way:
we need to extract the distances between the different cities.

One way in which we can provide these data is by asociating each node $\mathbf{i}$ with a distances matrix
$\mathcal{M}_{\mathbf{j},\mathbf{k}} = |\mathbf{i}(\mathbf{j}) - \mathbf{i}(\mathbf{k})|$
in which the cities are ordered according to the $\mathbf{i}$ index,
and by asociating each edge with a permutation matrix that corresponds to the permutation of that edge.
Then, to go from one node to another, following some edge,
we just have to conjugate the distances matrix of the source node $\mathcal{M}$
with the permutation matrix of the edge $\mathcal{G}$,
to obtain the distances matrix of the destination node $\mathcal{G} \mathcal{M} \mathcal{G}^T = \mathcal{N}$.
The $\mathcal{d}$ compass function in this case would be given by the tensor product
between the distances matrix of the node and the roller permutation.

[Here](Python%20-%20TSP%20circuit%20distance%20compass.ipynb) is some Python code
that demonstrates the ideas in the above discussion,
and shows how we can use the $\mathcal{d}$ function and arbitrary rollable sets of generators
to navigate the graph of a random TSP instance and find its sinks.

So, the local data for some node in these graphs would be distances matrices,
either just the distances matrix of the given node,
or in cojunction with the distances matrices of the nodes just one edge away from it
(i.e., in its immediate neighbourhood in the $\Omega$ graphs).

### Machine learning of TSP solutions

The question now is, can we use this local data to distinguish solution sinks from non solution sinks in $\Omega$ graphs for the TSP? And it seems we can, to a certain extent.

To train the ML models, we generate random instances of the TSP,
and we use the $\mathcal{d}$ compass functions
and appropriate generator sets
to find the sinks for each of them.
We assume that the one with the shortest circuit length is the solution.
Each sink corresponds to a distances matrix,
and has a number of incident edges connecting it to other nodes / distances matrices.

The data we obtain from the distances matrix associated to each sink are,
for each row (i.e., city) the circuit distance as if the row were a 1d sorting problem.
This is equivalent to the circuit length,
but expressed in $\mathbf{c}$ dimensions;
and we also add to the data for the node
the differences between its individual data
and the individual data of the adjacent nodes.

[Here](Python%20-%20TSP%20data.ipynb) is a demonstration of the Python code used to obtain this data.

And [here](Python%20-%20TSP%20model.ipynb) is a demonstration of the Python code
used to train the models with the data obtained,
and to test the trained models.

### Results

For instances of the TSP with 20 cities, and using aproximately 8.000 datapoints
(half of them correct solutions, and the other half non-solution sinks)
we obtain confusion matrices like
$\left[\begin{smallmatrix}61 & 39 \\\\ 26 & 74\end{smallmatrix}\right]$

And for 31 cities, using the same aproximate numbers of data, we obtain things like
$\left[\begin{smallmatrix}53 & 47 \\\\ 34 & 66\end{smallmatrix}\right]$

## next steps

### Computing power

We need more computing power than my humble laptop
to produce more data,
both for the numbers of cities that we have already studied above,
and for higher numbers.

### Shaking it

Given an instance of the TSP, i.e., a set $\mathbf{S} \subset \mathbb{Q}^2$ with $\mathbf{c} = |\mathbf{S}|$ and an index $\mathbf{i}$ on it,
we can obtain its distances matrix
$\mathcal{M}_{\mathbf{j},\mathbf{k}} = |\mathbf{i}(\mathbf{j}) - \mathbf{i}(\mathbf{k})|$.
And from this matrix we can obtain a 4d array of boolean values,
$\mathcal{B}$, with shape $\mathbf{c}^4$,
if which each entry $\mathcal{B}_{\mathbf{i},\mathbf{j},\mathbf{k},\mathbf{m}}$ is 1 in case
$\mathcal{M}_{\mathbf{i},\mathbf{j}} < \mathcal{M}_{\mathbf{k},\mathbf{m}}$
and 0 otherwise.

This allows us to establish an equivalence relation between instances of the TSP,
so that 2 instances are in the same class when they determine the same $\mathcal{B}$ matrix.
It is quite clear that any 2 instances of the TSP that belong to the same class,
will be equivalent wrt any algorithm we can devise to sort it according to any criterion;
effectively, this means that there are a finite number of TSP instances for each cardinality:
the number of different $\mathbf{c}^4$ arrays with entries in $\{0, 1\}$
that are consistent with a 2d distribution of points.

Now, let's note that if we take an instance of the TSP,
and slightly move one of its points (cities),
we will obtain another instance of the TSP,
which can be in the same class as the original,
or in another.
So there will be instances that are very near (as 2d distribiution of points)
to the instances of a different class,
and instances that sit squarely in the middle of their class,
and you need to displace any point quite a long way to get to another class.

The solution to these last instances should be more easily distinguishable
than the solutions to instances on the edge of their classes.
This is a conjecture.

So if the conjecture is correct,
we should be able to "shake" the instances,
i.e., randomly move its points without changing its class,
to obtain instances in the middle of their classes,
and with a more distinguishable solution.

Each of these classes should determine their own $\Omega$ graphs, however,
so in principle they should be equally navigable, so this is dubious.
And quite computationally costly...

But it would be interesting to calculate the different numbers of sinks for each class and generator set,
and with the $\mathcal{d}$ compass, at least for small numbers of cities.